In [ ]:
"""
Author: Wasif Rasool Qazi
Augumentation script for Sentinel-2-Dataset
"""


import os
import numpy as np
import rasterio
from PIL import Image
import albumentations as A

# Input directories
stackedImgDir = r"C:\Users\wasif\Desktop\Annotated Images\stacked"
maskDir = r"C:\Users\wasif\Desktop\Annotated Images\masks_resized"

# Output Directories
outputImgDir = r"C:\Users\wasif\Desktop\AugTest\images"
outputMaskDir = r"C:\Users\wasif\Desktop\AugTest\masks"
os.makedirs(outputImgDir, exist_ok=True)
os.makedirs(outputMaskDir, exist_ok=True)

# Use 5 Augumentation metric 
nAugs = 5

# Albumentations transformation
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.3),
    A.RandomRotate90(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=30, border_mode=0, p=0.4)
])

# Process all images
for file in os.listdir(stackedImgDir):
    # check if the file end with _stacked.jp2
    if file.endswith("_stacked.jp2"):
        baseName = file.replace("_stacked.jp2", "")
        stackedImgPath = os.path.join(stackedImgDir, file)
        # get masks corresponding to the stacked image 
        maskPath = os.path.join(maskDir, f"{baseName}_annotated_resized.png")

        if not os.path.exists(maskPath):
            print(f"Mask not found for {baseName}, skipping.")
            continue

        with rasterio.open(stackedImgPath) as src:
            imgStack = src.read()
            meta = src.meta

        imgStackNp = np.moveaxis(imgStack, 0, -1)
        maskNp = np.array(Image.open(maskPath).convert("L"))

        # Augumentation Loop
        for i in range(nAugs):
            augmented = transform(image=imgStackNp, mask=maskNp)
            augImg = np.moveaxis(augmented["image"], -1, 0)
            augMask = (augmented["mask"] > 127).astype(np.uint8) * 255

            # Check land and water pixels
            totalPixels = augMask.size
            landPixels = np.sum(augMask == 255)
            waterPixels = np.sum(augMask == 0)

            
            if landPixels / totalPixels >= 0.9:
                label = "land"
            elif waterPixels / totalPixels >= 0.9:
                label = "water"
            else:
                label = "mixed"

            # Output image and mask path , the names are kept same for easiness
            imgOutPath = os.path.join(outputImgDir, f"{baseName}_{label}_aug{i}.jp2")
            maskOutPath = os.path.join(outputMaskDir, f"{baseName}_{label}_aug{i}.png")

            # saving of aug masks and image
            try:
                with rasterio.open(
                    imgOutPath, "w",
                    driver="JP2OpenJPEG",
                    height=augImg.shape[1],
                    width=augImg.shape[2],
                    count=augImg.shape[0],
                    dtype=augImg.dtype,
                    crs=meta["crs"],
                    transform=meta["transform"]
                ) as dst:
                    dst.write(augImg)

                Image.fromarray(augMask).save(maskOutPath)

                print(f"Image: {imgOutPath} | Mask: {maskOutPath}")
            except Exception as e:
                print(f" Failed to save for {baseName}")

        print(f"Finished augmenting: {baseName}\n")
        print('Finished')


c:\Users\wasif\AppData\Local\Programs\Python\Python313\Lib\site-packages\PIL\Image.py:3402: DecompressionBombWarning: Image size (120560400 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Image: C:\Users\wasif\Desktop\AugTest\images\Image_10_mixed_aug0.jp2 | Mask: C:\Users\wasif\Desktop\AugTest\masks\Image_10_mixed_aug0.png
Image: C:\Users\wasif\Desktop\AugTest\images\Image_10_mixed_aug1.jp2 | Mask: C:\Users\wasif\Desktop\AugTest\masks\Image_10_mixed_aug1.png
Image: C:\Users\wasif\Desktop\AugTest\images\Image_10_mixed_aug2.jp2 | Mask: C:\Users\wasif\Desktop\AugTest\masks\Image_10_mixed_aug2.png
Image: C:\Users\wasif\Desktop\AugTest\images\Image_10_mixed_aug3.jp2 | Mask: C:\Users\wasif\Desktop\AugTest\masks\Image_10_mixed_aug3.png
Image: C:\Users\wasif\Desktop\AugTest\images\Image_10_mixed_aug4.jp2 | Mask: C:\Users\wasif\Desktop\AugTest\masks\Image_10_mixed_aug4.png
Finished augmenting: Image_10

Finished


In [3]:
import os
# --- Input Directories ---
imageDir = r"C:\Users\wasif\Desktop\AugTest\stacked"
maskDir = r"C:\Users\wasif\Desktop\AugTest\masks_resized"


imageFiles = sorted([f for f in os.listdir(imageDir) if f.endswith(".jp2")])
maskFiles = sorted([f for f in os.listdir(maskDir) if f.endswith(".png")])


for idx, (imgFile, maskFile) in enumerate(zip(imageFiles, maskFiles), start=61):
    ext = os.path.splitext(imgFile)[1]
    
    newImgName = f"stacked_{idx}{ext}"
    newMaskName = f"mask_{idx}.png"
    
    os.rename(os.path.join(imageDir, imgFile), os.path.join(imageDir, newImgName))
    os.rename(os.path.join(maskDir, maskFile), os.path.join(maskDir, newMaskName))
    
    print(f"Renamed: {imgFile} to {newImgName} and {maskFile} to {newMaskName}")


Renamed: stacked_61.jp2 to stacked_61.jp2 and mask_61.png to mask_61.png
Renamed: stacked_62.jp2 to stacked_62.jp2 and mask_62.png to mask_62.png
Renamed: stacked_63.jp2 to stacked_63.jp2 and mask_63.png to mask_63.png
Renamed: stacked_64.jp2 to stacked_64.jp2 and mask_64.png to mask_64.png
Renamed: stacked_65.jp2 to stacked_65.jp2 and mask_65.png to mask_65.png
Renamed: stacked_66.jp2 to stacked_66.jp2 and mask_66.png to mask_66.png
Renamed: stacked_67.jp2 to stacked_67.jp2 and mask_67.png to mask_67.png
Renamed: stacked_68.jp2 to stacked_68.jp2 and mask_68.png to mask_68.png
Renamed: stacked_69.jp2 to stacked_69.jp2 and mask_69.png to mask_69.png
Renamed: stacked_70.jp2 to stacked_70.jp2 and mask_70.png to mask_70.png
Renamed: stacked_71.jp2 to stacked_71.jp2 and mask_71.png to mask_71.png
Renamed: stacked_72.jp2 to stacked_72.jp2 and mask_72.png to mask_72.png
Renamed: stacked_73.jp2 to stacked_73.jp2 and mask_73.png to mask_73.png
Renamed: stacked_74.jp2 to stacked_74.jp2 and mask_